Threading + Multiprocessing Test
===

In [1]:
import multiprocessing as mp
import threading
import time

In [2]:
class ProducerProcess(mp.Process):
    def __init__(self, output_queue, **kwargs):
        super(ProducerProcess, self).__init__()
        self.output_queue = output_queue

    def run(self):
        print("ProducerProcess: started!")
        while True:
            time.sleep(5)  # do long computation
            print("ProducerProcess: adding to queue")
            self.output_queue.put(int(time.time()))

In [3]:
class ThreadsafeUpdatingDataContainer:
    def __init__(self, queue):
        self.data = None
        self.data_lock = threading.Lock()
        
        # start consumer thread to update the data from a queue
        consumer_thread = threading.Thread(
            name="Consumer thread", 
            target=self._update_data_from_queue, 
            args=(queue,),
            daemon=True
        )
        consumer_thread.start()

    def _update_data_from_queue(self, queue):
        while True:
            result = queue.get()  # this blocks this thread, but NOT the whole process
            with self.data_lock:
                print(f"Main process, consumer thread: received result '{result}'")
                self.data = result
    
    def get_data(self):
        with self.data_lock:
            return self.data


def run_test():
    queue = mp.Queue()
    
    producer_process = ProducerProcess(output_queue=queue, daemon=True)
    producer_process.start()

    data_container = ThreadsafeUpdatingDataContainer(queue)
    
    for i in range(10):
        time.sleep(2)  # this represents an I/O process that is occasionally querying the data
        print(f"Main process, main thread: Current data is '{data_container.get_data()}'")

In [4]:
run_test()

ProducerProcess: started!
Main process, main thread: Current data is 'None'
Main process, main thread: Current data is 'None'
ProducerProcess: adding to queue
Main process, consumer thread: received result '1603820572'
Main process, main thread: Current data is '1603820572'
Main process, main thread: Current data is '1603820572'
ProducerProcess: adding to queue
Main process, main thread: Current data is '1603820572'
Main process, consumer thread: received result '1603820577'
Main process, main thread: Current data is '1603820577'
Main process, main thread: Current data is '1603820577'
ProducerProcess: adding to queue
Main process, consumer thread: received result '1603820582'
Main process, main thread: Current data is '1603820582'
Main process, main thread: Current data is '1603820582'
Main process, main thread: Current data is '1603820582'
ProducerProcess: adding to queue
ProducerProcess: adding to queue
ProducerProcess: adding to queue
ProducerProcess: adding to queue


Process ProducerProcess-1:
Traceback (most recent call last):
  File "/home/levoniaz/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "<ipython-input-2-73a61a4ae76c>", line 9, in run
    time.sleep(5)  # do long computation
KeyboardInterrupt
